# RSA Algorithm

There are four steps when using RSA algorithm: keys generation, public key distribution, ciphering and deciphering.

The fundamental of RSA are three big positive an integer numbers, usually called "e", "d" and "n". The strength of RSA algorithm is the difficulty  of factoring the number "n" to obtain "e" and "d".

## Keys generation

To create the key pair, choose two prime numbers similars in magnitude but with little difference in quantity of digits to make factoring a hard task.

In example bellow we are using small numbers to reduce tutorial complexity, because we are interested in understanding the logic.

In [2]:
p = 5
q = 7

Next multiply these numbers to calculate "n". This number will be used as modulo to define the public and private keys.

In [3]:
n = p * q

print(n)

35


Now we need to calculte Phi. As "p" and "q" are prime numbers the Phi is defined as multiplication of p-1 and q-1

In [4]:
phi_of_n = (p - 1) * (q - 1)

print(phi_of_n)

24


Now let´s import some python functions...

First line imports the gcd function, which returns the greatest common divisor of two numbers. This function will be used to generate the private key.

The random function is used to generate a random number

In [5]:
from math import gcd
import random

Bellow we define the function to create the encrypton key. This key will be used to generate the private key

In [6]:
def get_encryption_key(n, phi_of_n):
    lst = [i for i in range(1, n+1)]
    e_list = []
    for i in lst:
        if (1 < i) and (i < phi_of_n):
            gcd_value = gcd(i, n)
            gcd_phi = gcd(i, phi_of_n)
            if (gcd_value == 1) and (gcd_phi == 1):
                e_list.append(i)
    if len(e_list) == 1:
        return e_list[0]
    else:
        return e_list[random.randint(1, len(e_list)-1)]  

This other function creates the decryption key, used to derive the public key

In [7]:
def get_decryption_key(e, phi_of_n):
    d_list = []
    for i in range(e * 25):
        if (e * i) % phi_of_n == 1:
            d_list.append(i)
    return d_list[random.randint(1, len(d_list) - 1)]

The code bellow uses these function to create the private and the public keys:

In [10]:
e = get_encryption_key(n, phi_of_n)
d = get_decryption_key(e, phi_of_n)

# Avoid colision
while d == e:
    d = get_decryption_key(e, phi_of_n)

public_key = [e, n]
private_key = [d, n]

print(public_key)
print(private_key)

[19, 35]
[235, 35]


## Public key distribution

Ideally the private key is created using HSM (Hardware Security Module) devices that don´t allow key extractionis and this key is never transmitted. When is necessary to transmit the private key, we need to use a security channel

## Message ciphering

First let´s define a function to return ASCII values from message characters:

In [9]:
import string
def text_to_digits(PT):
    pool = string.ascii_letters + string.punctuation + " "
    M = []
    for i in PT:
        M.append(pool.index(i))
    return M

And define a function to cipher the message using our public key:

In [11]:
def encrypt(M, public_key):
    return [(i ** public_key[0]) % public_key[1] for i in M]

Final step is to cipher the message using these functions

In [12]:
message = "My test"
ascii_message = text_to_digits(message)

cipher_message = encrypt(ascii_message, public_key)
print(cipher_message)

[17, 24, 14, 19, 4, 32, 19]


## Message deciphering

Similar of message ciphering, let´s define a function to return character from an ASCII value

In [13]:
def digits_to_text(DT):
    pool = string.ascii_letters + string.punctuation + " "
    msg = ''
    for i in DT:
        msg += pool[i]
    return msg

Now we define a function to decipher the message using private key:

In [14]:
def decrypt(CT, private_key):
    return [((i ** private_key[0]) % private_key[1]) for i in CT]

Final step is to use these functions to decipher the message:

In [15]:
ascii_decipher_message = decrypt(cipher_message, private_key) 
original_message = original_PT = digits_to_text(ascii_decipher_message)

print(original_message)

dyotest


We finished our example of RSA algorithm, generating key pair, ciphering and deciphering one message using these keys.

# Shor´s algorithm

Shor´s algorithm is a quantum computing algorithm capable of factoring an integer number, which is finding two numbers used in a multiplication to generate a third one.

This is the reverse path of RSA algorithm, which uses two prime numbers to generate the number "n". This number "n" is the base of this algorithm.

In the previous section we could understand that we need this number "n" to cipher and decipher messages. In other words, this number is known by all parts involved in message transmission.

In theory, to break the RSA encryption we need the number "n", a quantum computer with enough capacity to execute Shor´s algorithm  and the ciphered message.

In the beginning  of RSA section we defined the number "n". We will use Shor´s algorithm to calculate the factos of this number, which are the numbers needed to generate the public and private keys.

Shor´s algorithm is explained in diverse materials about quantum computing. In references section of this notebook you can find links explaning this algorithm.

Let´s use Qiskit and IBM quantum computer simulator to show how Shor´s algorithm to obtain numbers "p" and "q" that we defined in the beginning of RSA section to create the key pair.

First let´s import necessary functions:

In [16]:
import math
import numpy as np
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import Shor
from qiskit import IBMQ

Next step is to load our token from IBM Quantum Experience to execute our circuit in simulator. This token must by save in our computer previously.

In the code bellow we also get a backend instance of this simulator:

In [15]:
IBMQ.load_account()

provider = IBMQ.get_provider(group='open')
backend = provider.get_backend('ibmq_qasm_simulator')

To finish, let´s create the circuit which implements Shor´s algorithm to factor the number "n' using Qiskit functions.

Next we will execute the circuit obtaining "p" and "q" values, defined in the beginning of this notebook:

In [16]:
quantum_instance = QuantumInstance(backend, shots=30)
shor = Shor(quantum_instance=quantum_instance)
result = shor.factor(n)
print(f"Factors of \"n\"= {n} are {result.factors[0]}.")

Os fatores de "n" = 35 são [5, 7].


## Shor´s algorithm anatomy

In this section we create a circuit which implements Shor´s algorithm using Qiskit to verify the number of qubits needed for a quantum computer execute this circuit. This information is important to know the minimum  capacity of a quantum computer necessary to execute the circuit.

We also print the circuit to see the it three building blocks:


1. A block of Hadarmard (H gates) operations to put qubits in superposition

2. Factorization of number N

3. Quantum Fourier Transform (QFT) used to increase probability of correct answer through interference

In References section there are links with mathematical proof of each step and details about this algorithm

As we can confirm, at the moment there is no quantum computer with enough qubits to run Shor´s algorithm to break a real world communication. 

In [17]:
shor_circuit = shor.construct_circuit(15)

print(f"Quantity de qubits: {shor_circuit.num_qubits}")
shor_circuit.draw(output='mpl')

NameError: name 'shor' is not defined

We conclude this section with a practical example of Shor´s algorithm factoring number "n" of RSA algorithm.

With factos of "n" is possible to recreate the private and public keys used to cipher messages in a specific context. 

However, at the moment, there is no quantum computer with enough capacity to factor a real "n" in real world operations of communication or storage.

## Post quantum cryptography

What happen when quantum computers achieve enough capacity to break traditional algorithms? 

There under development post quantum cryptography algorithms resistant to quantum computers attacks. Recently NIST published the first list of these algorithms, there are some links about the subject in references section.

In future we should see these algorithms being used to cipher our data in communications and data storage. However, we should remember to change cipher of data at rest, like backups, from traditional algorithms to post quantum algorithms to avoid this kind of attack.

But this a topic for another text…


## References

RSA Algorithm: https://en.wikipedia.org/wiki/RSA_(cryptosystem)

RSA algorithm implementation: https://www.codespeedy.com/rsa-algorithm-an-asymmetric-key-encryption-in-python/#:~:text=RSA%20Algorithm%20working%20example&text=Compute%20totient%20%3D%20

Shor´s algorithm: https://en.wikipedia.org/wiki/Shor%27s_algorithm

Shor´r algorithm explanation: https://qiskit.org/textbook/ch-algorithms/shor.html

Qiskit documentation: https://qiskit.org/documentation/tutorials/algorithms/08_factorizers.html

IBM Quantum Experience home page: https://quantum-computing.ibm.com/

IBM Quantum Experience account configuration: https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq

Post quantum cryptographic links:
- https://www.nist.gov/news-events/news/2022/07/nist-announces-first-four-quantum-resistant-cryptographic-algorithms

- https://www.inovacaotecnologica.com.br/noticias/noticia.php?artigo=selecionados-algoritmos-criptografia-resistentes-computadores-quanticos&id=010150220708#.YtYdgKTQ9zB

- https://thequantuminsider.com/2022/07/11/crypto-quantique-announces-first-post-quantum-computing-iot-security-platform-compliant-with-new-nist-standards/
